In [1]:
import os, sys
import numpy as np
from random import shuffle
from math import log, floor
import pandas as pd
import tensorflow as tf
import tensorboard as tb
from keras import backend as K
from keras.models import *
from keras.layers import *
from keras.activations import *
from keras.callbacks import *
from keras.utils import *
from keras.layers.advanced_activations import *
from keras import *
from keras.engine.topology import *
from keras.optimizers import *
import keras
from sklearn.metrics import *
import glove
import gensim
from gensim.models.word2vec import *
from gensim.models.doc2vec import *
from keras.preprocessing.text import *
from keras.preprocessing.sequence import *
from keras.utils import *
from sklearn.model_selection import *
import random
from random import shuffle
import re
from collections import Counter
from keras.utils.generic_utils import *
from keras import regularizers
from keras.regularizers import l2
import string
import unicodedata as udata
import pickle
from keras.applications import *
from tqdm import *
# import tqdm
from keras.preprocessing.image import *
K.set_image_dim_ordering('tf')

/home/leoqaz12/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [47]:
# dict_ = pickle.load(open('data/call_seq.pkl','rb'))
# type(dict_)

pandas.core.series.Series

In [2]:
def tick_mapping(x):
    if  x.find('#')!=-1:
        return '#'
    else:
        return x

In [3]:
def split(x):
    if x.find('='):
        return x.lower().split('=')[-1]
    else:
        return x.lower()
        

In [4]:
def find_api(x):
#     if ['=','\\','/',':',';','-','<','.' ] not in x:
    if  x.find('=')==-1 and x.find('#')==-1 and x.find('/')==-1:
        x = re.sub('[^a-zA-Z]+','',x)
        return x
    else:
        return np.nan

In [5]:
def find_param(x):
    if  x.find('=')!=-1 and x.find('#')==-1 :
        return x.replace('=',' ').lower()
    else:
        return np.nan

In [742]:
# re.sub('^(A-Z+|a-z+|0-9+|_+)',' ','?gene%ric_write')
re.sub('[^0-9a-zA-Z]+',' ','?gene%ric_write')
# sting='?gene%ric_write'
# re.sub(r'^[a-zA-Z0-9][ A-Za-z0-9_-]*$',' ',sting.rstrip())

TypeError: decoding to str: need a bytes-like object, float found

In [6]:
def find_param_r(x):
    if  x.find('=')!=-1 and x.find('#')==-1 :
#         x = re.sub(r"([A-Z])",r" \1",x)
        x = re.sub(r"([A-Z][a-z])",r" \1",x)
        x = re.sub(r"([A-Z]+[ ])",r' \1',x) # 把大寫依照字拆開
        x = x.split('=')[-1].lower()
#         x = re.sub('_+|/+|-+|\.+|{+|}+|[+|]+|(+|)+|\\+|~+|^+|@+|#+|!+|%+|&+|\*+|\++|=+|?+',' ',x)
        x = re.sub("[^A-Za-z0-9]+",' ',x)
        return x
    else:
        return np.nan

In [7]:
def find_param_r_2(x):
    x = re.sub(r"([A-Z][a-z])",r" \1",x)
    x = re.sub(r"([A-Z]+[ ])",r' \1',x) # 把大寫依照字拆開
    x = x.split('=')[-1].lower()
    #         x = re.sub('_+|/+|-+|\.+|{+|}+|[+|]+|(+|)+|\\+|~+|^+|@+|#+|!+|%+|&+|\*+|\++|=+|?+',' ',x)
    x = re.sub("[^A-Za-z0-9]+",' ',x)
    return x

In [8]:
def find_param_l(x):
    if  x.find('=')!=-1 and x.find('#')==-1 :
        return x.split('=')[0].lower().strip()
    else:
        return np.nan

In [9]:
word_model_dict = pickle.load(open('model/word_model_dict.pkl','rb'))

In [10]:
def transform_malware_to_dict(s):
    """
    Transfer a malware to a list of dict.
    Input:
    - s: pandas.series, malware
    
    Output: a list of malware attributes
    """
    list_ = []
    temp_dict = {}
    for k in s:
        if k == '#':
            if bool(temp_dict):
                list_.append(temp_dict)
            temp_dict = {}
        elif k.find('=') == -1:
            temp_dict['api'] = k
        else:
            sp = k.split('=')
            temp_dict[sp[0]] = sp[1] if sp[1] else ''
#     print(list_)
    return list_
def transform_api_to_feature(d):
    """
    Transform a dict of api to trainable features.
    Input:
    - d: dict, api keys and values
    - word_model_dict: a dict of trained doc2vec model
    - dim_dict: a dict of doc2vec dimention
    
    Output: tranable features np.array
    """
    hex2dec_list = ['lpcbData', 'lpdwDisposition', 'phkresult', 'samDesired']
    special_list = ['Return', 'uExitCode', 'lpData', 'lpType']
    word_vector_list = ['dwDesiredAccess', 'dwShareMode', 'hkey', 'hObject', 'lpFileName', 
                        'lpSubKey', 'lpValueName', 'procName', 'dwCreationDisposition', 'dwType']
    dec_list = ['dwFlags', 'dwFlagsAndAtrributes', 'dwIndex']
    
    dim_dict = {
        'dwDesiredAccess':10,
        'dwShareMode':3,
        'hkey':15,
        'hObject':4,
        'lpFileName':5,
        'lpSubKey':4,
        'lpValueName':3,
        'procName':2, 
        'dwCreationDisposition':3, 
        'dwType':2
    }
    
    output_features = []
    
    for key in hex2dec_list:
        if key in d.keys():
            f = int(d[key], 16)
        else:
            f = -1
        output_features.append([f])
        
    for key in word_vector_list:
        if key in d.keys():
            f = word_model_dict[key.lower()].infer_vector([d[key]]).tolist()
        else:
            f = [-1] * dim_dict[key]
        output_features.append(f)
    
    for key in dec_list:
        if key in d.keys():
            try:
                f = float(d[key])
            except:
                f = int(d[key], 16)
        else: 
            f = -1
        output_features.append([f])
        
    key = 'Return'
    if key in d.keys():
        if d[key].lower() == 'success':
            f = 1
        elif d[key].lower() == 'failure':
            f = 0
        else:
            f = -1
    else:
        f = -1
    output_features.append([f])
    
    key = 'uExitCode'
    if key in d.keys():
        if int(d[key]) > 255/2:
            f = 1
        else:
            f = 0
    else:
        f = -1
    output_features.append([f])
    
    key = 'lpData'
    if key in d.keys():
        f = []
        f.append(len(d[key]))
        f.append(float(d[key].replace('.', '', 1).isdigit()))
    else:
        f = [-1, -1]
    output_features.append(f)
    
    key = 'lpType'
    if key in d.keys():
        f = [0] * 5
        label = ['0', 'REG_DWORD', 'REG_BINARY', 'REG_SZ', 'REG_EXPAND_SZ']
        for i in range(len(label)):
            if d[key] == label[i]:
                f[i] = 1
    else:
        f = [-1] * 5
    output_features.append(f)
    
    key='api'
    if key in d.keys():
        f = [0] * 32
        label = ['WinHttpWriteData','GetUrlCacheEntryInfo', 'TerminateProcess','OpenThread','RegDeleteKey',
'HttpSendRequest','RegOpenKey','LoadLibrary','RegQueryValue','RegCloseKey', 'RegCreateKey',
 'CreateFile', 'CloseHandle', 'RegOpenCurrentUser', 'RegEnumValue', 'CreateRemoteThread',
 'CreateThread',  'DeleteFile', 'RegSetValue', 'ExitProcess', 'OpenProcess', 'CopyFile',
 'RegDeleteValue', 'CreateProcess', 'WinHttpOpen', 'WinHttpConnect', 'WinHttpOpenRequest',
 'WinHttpSendRequest',  'WinHttpReadData', 'InternetOpen', 'InternetConnect', 'WinExec']
        for i in range(len(label)):
            if d[key] == label[i]:
                f[i] = 1
    else:
        f = [-1] * 32
    output_features.append(f)
#     print(np.array(output_features))
    return np.concatenate(output_features)

In [750]:
def encode_api(d):
    output_features = []
    key='api'
    if key in d.keys():
        f = [0] * 32
        label = ['WinHttpWriteData','GetUrlCacheEntryInfo', 'TerminateProcess','OpenThread','RegDeleteKey',
'HttpSendRequest','RegOpenKey','LoadLibrary','RegQueryValue','RegCloseKey', 'RegCreateKey',
 'CreateFile', 'CloseHandle', 'RegOpenCurrentUser', 'RegEnumValue', 'CreateRemoteThread',
 'CreateThread',  'DeleteFile', 'RegSetValue', 'ExitProcess', 'OpenProcess', 'CopyFile',
 'RegDeleteValue', 'CreateProcess', 'WinHttpOpen', 'WinHttpConnect', 'WinHttpOpenRequest',
 'WinHttpSendRequest',  'WinHttpReadData', 'InternetOpen', 'InternetConnect', 'WinExec']
        for i in range(len(label)):
            if d[key] == label[i]:
                f[i] = 1
    else:
        f = [-1] * 32
    output_features.append(f)
    return np.concatenate(output_features)

In [11]:
def clean_values(d):
    a={}
    for key, value in d.items():
        if key == 'api':
            a[key] =  re.sub('[^a-zA-Z]+','',value)
#             continue
        else:
            d[key] = find_param_r_2(value)
    return d

In [747]:
# label = ['WinHttpWriteData','GetUrlCacheEntryInfo', 'TerminateProcess','OpenThread','RegDeleteKey',
# 'HttpSendRequest','RegOpenKey','LoadLibrary','RegQueryValue','RegCloseKey', 'RegCreateKey',
#  'CreateFile', 'CloseHandle', 'RegOpenCurrentUser', 'RegEnumValue', 'CreateRemoteThread',
#  'CreateThread',  'DeleteFile', 'RegSetValue', 'ExitProcess', 'OpenProcess', 'CopyFile',
#  'RegDeleteValue', 'CreateProcess', 'WinHttpOpen', 'WinHttpConnect', 'WinHttpOpenRequest',
#  'WinHttpSendRequest',  'WinHttpReadData', 'InternetOpen', 'InternetConnect', 'WinExec']
# len(label)

32

In [12]:
# a = np.array([[1, 2, 3]])
df = pd.read_csv('data/train_label_10.csv',header=None)
df2 = pd.read_csv('data/test_label_10.csv',header=None)
df.drop([0],inplace=True)
df2.drop([0],inplace=True)
train_list = df[0].values.tolist()
test_list = df2[0].values.tolist()
# '2b50a852dc3e3a389b31c920785a2c73c9c614165c2bc578112675d9542e5d23_3364.trace.hooklog'.split('.')[0] in train_list

In [781]:
# a = np.concatenate((a, np.array([1, 2, 3])), axis=0)
pickle.dump(word_model_dict,open('model/word_model_dict.pkl','wb'))

In [336]:
# "A_B/B".replace('_|/',' ')
kk = re.sub(r"([A-Z][a-z])",r" \1",'APPLEBanANaBALion')
re.sub(r"([A-Z]+[ ])",r' \1',kk)

' APPLE Ban A Na BA Lion'

## training sen2vec - prepare data

In [15]:
pad_len = 555
train_test = train_list
df_all = pd.DataFrame( columns=['api'])
df2 = pd.DataFrame( columns=['param_left','param_right'])
df_all2 = pd.DataFrame( columns=['param_left','param_right'])
all_feature = np.empty((0,99))
all_features = np.empty((0,pad_len,99))
x = np.array([0]*99)
kk = pd.Series([x])
for name in tqdm(os.listdir('data/trace_by_family_5up/')):
    for hl in os.listdir('data/trace_by_family_5up/'+name):
#         print(name,hl) #debug用
        if hl.split('.')[0] not in train_test:  #train_list/test_list
            continue
        try:
            df = pd.read_csv("data/trace_by_family_5up/"+name+"/"+hl ,header=None,encoding='latin1',sep='\n')
        except:
            df = pd.read_table("data/trace_by_family_5up/"+name+"/"+hl ,header=None,encoding='latin1')
        df = df.drop(df.iloc[0].index,axis=0)
        df[0] = df[0].map(tick_mapping)
#         print(len(df[0]))
        malware_dict_list = transform_malware_to_dict(df[0])
#         print(len(malware_dict_list))
        malware_dict_list = pd.Series(data=malware_dict_list).map(clean_values)
#         api_call = api_call.map(encode_api)
        malware_dict_list = malware_dict_list.map(transform_api_to_feature)
        api_length = len(malware_dict_list)
        if api_length <pad_len:  #pad size
            for i in range(pad_len-api_length):
                malware_dict_list = malware_dict_list.append(kk)
#                 print(api_length)
        else:
            malware_dict_list = malware_dict_list[:pad_len]
#         break
#         print(len(malware_dict_list))

        features = malware_dict_list.values#.tolist()
#         feature_a = api_call.values
#         print(features.shape,feature_a.shape)
        all_feature = np.append(all_feature, features)
#         n_pad = 1000 - len(all_feature) #pad number
#         all_feature = np.pad(all_feature, pad_width=n_pad , mode='constant',constant_values=-1)[n_pad:]
#         df['api'] = df[0].map(find_api)
#         df['param'] = df[0].map(find_param)
#         df2['param_left'] = df[0].map(find_param_l)
#         df2['param_right'] = df[0].map(find_param_r)
#         df_api = pd.DataFrame(df.api)
#         df_api = df_api.dropna()
#         df2 = df2.dropna()
#         df_api.reset_index(inplace=True,drop=True)
#         list_ = df.param.str.cat(sep=' ',na_rep='##').split('## ## ')
#         list_.pop(0)
#         len(list_) #取出單支malware param 資料
#         df_con = pd.merge(df_api,pd.DataFrame(list_),how='inner',right_index=True,left_index=True)
#         df_con.columns = ['api','param']
#         df_all = df_all.append(df_con,ignore_index=True)
#         df_all = df_all.append(df_api,ignore_index=True)
    
        all_feature = np.stack(all_feature,axis=0)
        all_feature = np.expand_dims(all_feature,axis=0)
#         print(all_feature.shape, all_features.shape)
        all_features = np.append(all_features,all_feature,axis=0)
        all_feature = np.empty((0,99))
# df_all2.head()
# all_features.shape

100%|██████████| 52/52 [24:26<00:00, 28.20s/it]


***

In [941]:
def pad_along_axis(array: np.ndarray, target_length, axis=0):

    pad_size = target_length - array.shape[axis]
    axis_nb = len(array.shape)

    if pad_size < 0:
        return a

    npad = [(0, 0) for x in range(axis_nb)]
    npad[axis] = (0, pad_size)

    b = np.pad(array, pad_width=npad, mode='constant', constant_values=0)

    return b

In [939]:
all_feature
n_pad = 1000 - len(all_feature)
kk = np.pad(all_feature,pad_width=n_pad, mode='constant',constant_values=0)[n_pad:]
kk.shape

(1000,)

In [1005]:
malware_dict_list.append(kk)

0      [-1.0, -1.0, 1243044.0, 131097.0, -1.0, -1.0, ...
1      [-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....
2      [-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....
3      [-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....
4      [-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....
5      [-1.0, -1.0, 1907552560.0, 131097.0, -1.0, -1....
6      [-1.0, -1.0, 1244420.0, 33554432.0, -1.0, -1.0...
7      [1244388.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0...
8      [-1.0, -1.0, 1244372.0, 33554432.0, -1.0, -1.0...
9      [1244332.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0...
10     [1244260.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0...
11     [-1.0, -1.0, 1244288.0, 33554432.0, -1.0, -1.0...
12     [1244340.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0...
13     [1244340.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0...
14     [-1.0, -1.0, 1244384.0, 33554432.0, -1.0, -1.0...
15     [-1.0, -1.0, 1244312.0, 131097.0, -1.0, -1.0, ...
16     [1244280.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0...
17     [1244248.0, -1.0, -1.0, 

In [945]:
all_feature.shape
kk = pad_along_axis(all_feature,1000,axis=0)
kk.shape

(1000,)

In [978]:
k =  pd.DataFrame(np.array([0]*99)).T
k

,0,1,2,3,4,5,6,7,8,9,...,89,90,91,92,93,94,95,96,97,98
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [986]:
x = np.array([-1]*99)
kk = pd.Series([x])

In [745]:
# features[4].shape ,len(features)
# print(features[0])
# features[1]
# features
# all_features.shape
df_all['api'].unique().tolist()
# df_api
# output_features
# malware_dict_list
# malware_dict_list
# np.stack(all_feature,axis=0)


['RegOpenKey',
 'LoadLibrary',
 'RegQueryValue',
 'RegCloseKey',
 'RegCreateKey',
 'CreateFile',
 'CloseHandle',
 'RegOpenCurrentUser',
 'RegEnumValue',
 'CreateRemoteThread',
 'CreateThread',
 'DeleteFile',
 'RegSetValue',
 'ExitProcess',
 'OpenProcess',
 'CopyFile',
 'RegDeleteValue',
 'CreateProcess',
 'WinHttpOpen',
 'WinHttpConnect',
 'WinHttpOpenRequest',
 'WinHttpSendRequest',
 'WinHttpReadData',
 'InternetOpen',
 'InternetConnect',
 'WinExec',
 'HttpSendRequest',
 'RegDeleteKey',
 'OpenThread',
 'TerminateProcess',
 'GetUrlCacheEntryInfo',
 'AAADZgAAqAAAABAAAADCwpRdKLuNdNg94Vgx4DsCAAAAAASAAACgAAAAEAAAAKTA',
 'ZgaWCgl7kpNPhIBoznsIAAAASW3TjYZljOQUAAAAhQ500JoiPBl8vSJYFdPwlBhc',
 'AcceptEncodinggzipdeflate',
 '',
 'ProxyauthorizationBasic',
 'headtitle301MovedPermanently',
 'TitleKeywordsandDescription',
 'WinHttpWriteData',
 'query',
 'cit']

In [646]:
# features = np.reshape(features,(5789,67))
np.squeeze(all_features,axis=-1)

ValueError: cannot select an axis to squeeze out which has size not equal to one

In [440]:
i = 7
df_all_2_sort = df_all2.sort_values(by='param_left')
key = key_list[i]
print(key)
# df_all_2_sort[df_all_2_sort['param_left'] == key]['param_right'].value_counts()
df_all_2_sort[df_all_2_sort['param_left'] == key]['param_right'].unique()
# len(df_all2['param_left'].unique())

procname


array(['winlogon exe', 'rundll32 exe', 'svchost exe', 'malware exe',
       'wmiadap exe', 'services exe', 'wmiprvse exe', 'alg exe'],
      dtype=object)

In [463]:
bool({'45':45})

True

In [694]:
kk = df[0].map(tick_mapping)
kk
    

1                                                      #
2                                             RegOpenKey
3                                hKey=HKEY_LOCAL_MACHINE
4                                       samDesired=20019
5                                       phkresult=12f7a4
6                                            ulOptions=0
7      lpSubKey=Software\Microsoft\Windows NT\Current...
8                                                  EAX=2
9                                               Return=2
10                                                     #
11                                           LoadLibrary
12              lpFileName=C:\WINDOWS\system32\IMM32.DLL
13                                             dwFlags=0
14                                          EAX=76390000
15                                        Return=SUCCESS
16                                                     #
17                                           LoadLibrary
18                             

In [469]:
list_

[{'key': 'CloseHandle', 'hObject': '1f8', 'EAX': '1', 'Return': 'SUCCESS'},
 {'key': 'RegQueryValue',
  'hKey': '54',
  'lpValueName': 'EnableAutodial',
  'lpcbData': '1f9f89c',
  'EAX': '2',
  'Return': '2',
  'lpType': '1f9f8ac',
  'lpData': '1f9f8a8'},
 {'key': 'RegCreateKey',
  'hKey': 'HKEY_LOCAL_MACHINE',
  'lpSubKey': 'System\\CurrentControlSet\\Services\\Tcpip\\Parameters',
  'lpClass': 'Class',
  'dwOptions': '0',
  'samDesired': '20019',
  'lpSecurityAttributes': '0',
  'phkresult': '32ff7d4',
  'lpdwDisposition': '731fcf40',
  'EAX': '0',
  'Return': '0'},
 {'key': 'RegOpenKey',
  'hKey': 'HKEY_LOCAL_MACHINE',
  'samDesired': '20019',
  'phkresult': '32ff804',
  'ulOptions': '0',
  'lpSubKey': 'System\\CurrentControlSet\\Services\\DnsCache\\Parameters',
  'EAX': '0',
  'Return': '0'},
 {'key': 'RegOpenKey',
  'hKey': 'HKEY_LOCAL_MACHINE',
  'samDesired': '20019',
  'phkresult': '32ff7f8',
  'ulOptions': '0',
  'lpSubKey': 'Software\\Policies\\Microsoft\\Windows NT\\DnsClient

In [484]:
df_all_2_sort.param_left.unique()

array(['binherithandle', 'binherithandles', 'dwcopyflags',
       'dwcreationdisposition', 'dwcreationflags', 'dwdesiredaccess',
       'dwflags', 'dwflagsandattributes', 'dwindex', 'dwoptions',
       'dwprocessid', 'dwsharemode', 'dwstacksize', 'dwtype', 'eax',
       'hkey', 'hobject', 'hprocess', 'htemplatefile', 'lpcbdata',
       'lpcbvalue', 'lpcchvaluename', 'lpclass', 'lpdata',
       'lpdwdisposition', 'lpenvironment', 'lpexistingfilename',
       'lpfilename', 'lpparameter', 'lpprocessattributes',
       'lpprocessinformation', 'lpsecurityattributes', 'lpstartaddress',
       'lpstartupinfo', 'lpsubkey', 'lpthreadattributes', 'lpthreadid',
       'lptype', 'lpvalue', 'lpvaluename', 'pbcancel', 'phkresult',
       'procname', 'return', 'samdesired', 'uexitcode', 'uloptions'],
      dtype=object)

In [493]:
df_all_2_sort = df_all2.sort_values(by='param_left')

# key_list = ['dwdesiredaccess','dwsharemode','hkey','hobject','lpfilename','lpsubkey','lpvaluename','procname']
dim_dict = {
    'dwdesiredaccess':10,
    'dwsharemode':3,
    'hkey':15,
    'hobject':4,
    'lpfilename':5,
    'lpsubkey':4,
    'lpvaluename':3,
    'procname':2, 
    'dwcreationdisposition':3, 
    'dwtype':2, 
    'lptype':5
}
key_list = dim_dict.keys()
word_model_dict = {}
for key in key_list:
    docs = []
    analyzedDocument = namedtuple('AnalyzedDocument', 'words tags')
    sent_list = df_all_2_sort[df_all_2_sort['param_left'] == key]['param_right'].values.tolist() + ['unk']
    for i, text in enumerate(sent_list):
        words = text.split()
        tags = [i]
        docs.append(analyzedDocument(words, tags))
    model = Doc2Vec(docs, vector_size=dim_dict[key], min_count=0, workers=dim_dict[key],window=dim_dict[key])
    word_model_dict[key] = model

In [260]:
df_all_2_sort[df_all_2_sort['param_left'] == 'dwdesiredaccess']['param_right'].values[0]

' eneric_read '

In [444]:
word_model_dict['hkey'].infer_vector(['ll'])

array([-0.00486637, -0.03297098, -0.00904916,  0.02176848,  0.02906333,
        0.02025512, -0.0325756 ,  0.0011921 ,  0.01728248,  0.02274979,
       -0.00524263,  0.02784625,  0.0099521 ,  0.00743065,  0.00179281],
      dtype=float32)

In [140]:
kk = 'i/love/u'.replace('/',' ')
# ''.join(map(str, kk))
kk

'i love u'

In [403]:
# word_test = Doc2Vec(['it is so cool s'.split(),'oh what a nice day'.split(),'im csie ee'.split()], size=2, min_count=0, workers=10, window=5)
docs = []
analyzedDocument = namedtuple('AnalyzedDocument', 'words tags')
doc1 = ["This is a sentence", "This is another sentence"]
for i, text in enumerate(doc1):
    words = text.lower().split()
    tags = [i]
    docs.append(analyzedDocument(words, tags))
word_test = Doc2Vec( docs, vector_size=2, min_count=0, workers=10, window=5)

In [433]:
# word_test.most_similar('this is')
word_test.infer_vector(['THis is a sentence'])

array([ 0.14136183, -0.05762891], dtype=float32)

 #### each malware 

In [118]:
df = pd.read_csv('data/trace_by_family_5up/jeefo/0a599da304a3dfa55b75f1f66113ab6d00a2465a8a208a16080ba9b636e301f4_2972.trace.hooklog'
                 ,header=None,sep='\n')
df = df.drop(df.iloc[0].index,axis=0)
df['api'] = df[0].map(find_api)
df['param'] = df[0].map(find_param)
df

,0,api,param
1,#391150000,NaN,NaN
2,LoadLibrary,loadlibrary,NaN
3,lpFileName=C:\WINDOWS\system32\IMM32.DLL,NaN,lpfilename c:\windows\system32\imm32.dll
4,dwFlags=0,NaN,dwflags 0
5,EAX=76390000,NaN,eax 76390000
6,Return=SUCCESS,NaN,return success
7,#392770000,NaN,NaN
8,LoadLibrary,loadlibrary,NaN
9,lpFileName=gdi32.dll,NaN,lpfilename gdi32.dll
10,dwFlags=0,NaN,dwflags 0


In [97]:
df_api = pd.DataFrame(df.api)
df_api = df_api.dropna()
df_api.reset_index(inplace=True,drop=True)
df_api #f取出單支malware api資料

,api
0,loadlibrary
1,loadlibrary
2,loadlibrary
3,loadlibrary
4,regopenkey
5,regopenkey
6,regqueryvalue
7,regclosekey
8,regopenkey
9,regqueryvalue


In [110]:
list_ = df.param.str.cat(sep=' ',na_rep='##').split('## ## ')
list_.pop(0)
len(list_) #取出單支malware param 資料
df_con = pd.merge(df_api,pd.DataFrame(list_),how='inner',right_index=True,left_index=True) #合併API、param
df_con

,api,0
0,loadlibrary,lpfilename c:\windows\system32\imm32.dll dwfla...
1,loadlibrary,lpfilename gdi32.dll dwflags 0 eax 77f10000 re...
2,loadlibrary,lpfilename gdi32.dll dwflags 0 eax 77f10000 re...
3,loadlibrary,lpfilename lpk.dll dwflags 0 eax 629c0000 retu...
4,regopenkey,hkey hkey_local_machine samdesired 20019 phkre...
5,regopenkey,hkey hkey_local_machine samdesired f003f phkre...
6,regqueryvalue,hkey hkey_local_machine\system\currentcontrols...
7,regclosekey,hkey hkey_local_machine\system\currentcontrols...
8,regopenkey,hkey hkey_local_machine samdesired 20019 phkre...
9,regqueryvalue,hkey hkey_local_machine\system\currentcontrols...


In [29]:
# for name in os.listdir('data/trace_by_family_5up/'):
#     for hl in os.listdir('data/trace_by_family_5up/'+name+'/'):
#         print(name,hl)
#         df = pd.read_table("data/trace_by_family_5up/"+name+"/"+hl, header=None,encoding='latin1' ,names=['Motif','num','Tick','Api'])
#         df2 = pd.DataFrame( columns=['main','sub'])

# #         try:
#         df2['main'], df2['sub'] = df['Api'].str.split('\n', 1).str
# #         except Exception as e:
# #             print("======"+str(e)+"========")
# #             continue
#         df2['sub'] = df2['sub'].str.replace('\n','=')  ##MAIN或SUB是否要把大小寫拆開成兩個字?

#         df2['sub'] = df2['sub'].map(lambda x: leo_split(str(x)))#.split('=')[1:])

#         df3 = df3.append({'main':df2.main.str.cat(sep=" "),'sub':df2['sub'].str.cat(sep=" "),'label':name},ignore_index=True)
# # df3 = df3.append({'sub':df2['sub'].str.cat(sep=" ")},ignore_index=True)

klez 0e2c1db70d7a5cadf48450a335801b4ff3fe59c9a515b86991a8e330d4d0f8dd_2952.trace.hooklog


AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas

NameError: name 'df3' is not defined

***

In [24]:
all_features.shape
train_X = all_features
test_X = test_X.reshape(422,150,99)

## Model

In [17]:
train_X = all_features#.reshape(3798,-1)
train_X.shape

(3798, 555, 99)

In [14]:
test_X = all_features#.reshape(422,-1)
test_X.shape
# all_features.shape

(422, 555, 99)

In [21]:
train = pd.read_csv('data/train_label_10.csv',header=None)
train = train.drop([0])
train = train.drop([0],axis=1)
train.reset_index(drop=True,inplace=True)
train_Y = np.array(train).astype('float')
train_Y.shape

(3798, 46)

In [22]:
test = pd.read_csv('data/test_label_10.csv',header=None)
test = test.drop([0])
test = test.drop([0],axis=1)
test.reset_index(drop=True,inplace=True)
test_Y = np.array(test).astype('float')
test_Y.shape

(422, 46)

In [23]:
np.save('data/train_X_10_555.npy',train_X)
np.save('data/test_X_10_555.npy',test_X)

# train_X = np.load('data/train_X_10.npy')
# test_X = np.load('data/test_X_10.npy')

In [24]:
temp = np.bincount(train_Y.reshape((-1,)).astype('int'))
weight = 1/temp*temp[0]
weight

array([ 1.       , 17.9549745])

In [25]:
def mcor(y_true, y_pred):
    #matthews_correlation
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos


    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos


    tp = K.sum(y_pos * y_pred_pos)
    tn = K.sum(y_neg * y_pred_neg)


    fp = K.sum(y_neg * y_pred_pos)
    fn = K.sum(y_pos * y_pred_neg)


    numerator = (tp * tn - fp * fn)
    denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))


    return numerator / (denominator + K.epsilon())




def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall))

def weighted_mse(y_true,y_pred):
    return (y_true*weight[1] + (1 - y_true)) * K.square(y_true-y_pred)

In [47]:
train_X = train_X.reshape(3798,-1)
test_X = test_X.reshape(422,-1)
train_X.shape

(3798, 14850)

In [26]:
lmbda = 0.00001
model = Sequential()
# model.add(Embedding(input_dim=14850,output_dim=256,embeddings_regularizer=l2(lmbda),mask_zero = True))
model.add(LSTM(1024,dropout=0.6,input_shape=(555,99),return_sequences=True,recurrent_dropout=0.6,kernel_initializer='lecun_normal'))
model.add(LSTM(1024,dropout=0.6,return_sequences=True,recurrent_dropout=0.6,kernel_initializer='lecun_normal'))
model.add(LSTM(1024,dropout=0.6,return_sequences=False,recurrent_dropout=0.6,kernel_initializer='lecun_normal'))
model.add(BatchNormalization())
model.add(Dense(512,activation='selu',kernel_initializer='lecun_normal'))
model.add(Dropout(0.6))
model.add(BatchNormalization())
model.add(Dense(512,activation='selu',kernel_initializer='lecun_normal'))
model.add(Dropout(0.6))
model.add(BatchNormalization())
model.add(Dense(256,activation='selu',kernel_initializer='lecun_normal'))
model.add(Dropout(0.6))
model.add(BatchNormalization())
model.add(Dense(256,activation='selu',kernel_initializer='lecun_normal'))
model.add(Dropout(0.6))
model.add(BatchNormalization())
model.add(Dense(46,activation='sigmoid',kernel_initializer='lecun_normal'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 555, 1024)         4603904   
_________________________________________________________________
lstm_5 (LSTM)                (None, 555, 1024)         8392704   
_________________________________________________________________
lstm_6 (LSTM)                (None, 1024)              8392704   
_________________________________________________________________
batch_normalization_6 (Batch (None, 1024)              4096      
_________________________________________________________________
dense_6 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 512)               2048      
__________

In [ ]:
model.compile(optimizer='adam',loss=weighted_mse,metrics=[f1])

batchSize=128
patien=15
epoch=350
saveP = 'model/naive555_10.h5'

callback=[
EarlyStopping(patience=patien,monitor='val_loss',verbose=1,mode='min'),
ModelCheckpoint(saveP,monitor='val_f1',verbose=1,save_best_only=True, save_weights_only=True,mode='max'),
]
model.fit(train_X, train_Y,
                epochs=epoch,
                batch_size=batchSize,
                shuffle=True,
                validation_data=(test_X,test_Y),
                callbacks=callback, 
                class_weight='auto'
                )

Train on 3798 samples, validate on 422 samples
Epoch 1/350
3798/3798 [==============================] - 115s 30ms/step - loss: 0.5210 - f1: 0.1007 - val_loss: 0.4438 - val_f1: 0.1961

Epoch 00001: val_f1 improved from -inf to 0.19605, saving model to model/naive555_10.h5
Epoch 2/350
3798/3798 [==============================] - 108s 28ms/step - loss: 0.4800 - f1: 0.1204 - val_loss: 0.4109 - val_f1: 0.1843

Epoch 00002: val_f1 did not improve from 0.19605
Epoch 3/350
3798/3798 [==============================] - 108s 28ms/step - loss: 0.4209 - f1: 0.1555 - val_loss: 0.3411 - val_f1: 0.2996

Epoch 00003: val_f1 improved from 0.19605 to 0.29962, saving model to model/naive555_10.h5
Epoch 4/350
3798/3798 [==============================] - 108s 28ms/step - loss: 0.3568 - f1: 0.2127 - val_loss: 0.3141 - val_f1: 0.2875

Epoch 00004: val_f1 did not improve from 0.29962
Epoch 5/350
3798/3798 [==============================] - 108s 28ms/step - loss: 0.3084 - f1: 0.2702 - val_loss: 0.2768 - val_f1:

In [42]:
model.evaluate(test_X,test_Y)
# ans = model.predict(test_X)
# ans[0]

422/422 [==============================] - 1s 3ms/step


[0.26503660152026265, 0.34885976580081957]

NameError: name 'ans' is not defined

In [37]:
ans[309] >= 0.5

array([ True, False, False, False, False, False, False, False, False,
        True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
        True,  True,  True, False, False, False,  True, False, False,
       False])

In [814]:
model.save('model/20180528.h5')

In [20]:
train_Y = train_Y.astype('float')

array([ 1.       , 17.9549745])